<a href="https://colab.research.google.com/github/mogollonalex/Econometrics-R/blob/main/Econometrics_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#     i:,:,,:i                                                              
#    ,@B@B@@@@                                                              
# iE2@G0qENEqMOFP                                           J       :,      
# NB@q        @B@,  rr7:  ir7,  vi7:  :r7i  rrr::rr   i;r. .@;  7r: i  :i:, 
# u@BB2SSPSX5kB@B. vL  i ru  G .8  E. G  1i M. X7 i5 7L ,M  N  :N   k. ::iM 
# FB@BEGOOOOOEF58  Ji    77  N .1  1: Z  77 0  L: .F LX:i.  G  :L   P ,F  Z 
# 1@B@...,.,...,:  .Jvvv  Y77v .7  v. 77ru  Y  r, .v  vrr:  ;j .;   L  jiiX.
#    U@@@B@@@@@B@:                                                          
#     kJvLvLLLvuP                                                           
                                                                               
####################################################################################################
############                   Elaborado por W. Alexander Mogollón B.           ####################
####################################################################################################

####################################################################################################
############                              Econometría con R                     ####################
####################################################################################################


##Preparando el entorno

### Instalación de las librerías necesarias

In [ ]:
install.packages(c("car","TSA","tseries","lmtest","fBasics","rgl",
                  "forecast","openxlsx","timsac","strucchange",
                  "sandwich","nlme","foreign","gplots","plm"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘xts’, ‘TTR’, ‘bitops’, ‘miscTools’, ‘rbibutils’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘leaps’, ‘locfit’, ‘quadprog’, ‘zoo’, ‘quantmod’, ‘timeDate’, ‘timeSeries’, ‘gss’, ‘stabledist’, ‘htmlwidgets’, ‘fracdiff’, ‘Rcpp’, ‘urca’, ‘RcppArmadillo’, ‘gtools’, ‘caTools’, ‘bdsmatrix’, ‘collapse’, ‘maxLik’, ‘Rdpack’, ‘Formula’


Warning message in install.packages(c("car", "TSA", "tseries", "lmtest", "fBasics", :
“installation of package ‘collapse’ had non-zero exit status”
Warning message in install.packages(c("car", "TSA", "tseries", "lmtest", "fBasics", :
“installation of package ‘rgl’ had non-zero exit status”
Warning message in install.packages(c("car", "TSA", "tseries", "lmtest", "fBasics", :
“installation of package ‘openxlsx’ had non-zero exit status”


Notas personales sobre la liberia en función a la utilidad

In [ ]:
### Factor inflador de varianza (multicolinealidad)
library(car)
###Heterocedásticidad 
library(lmtest)
### MCG
library(nlme)

###Importamos las liberías correspondientes

In [ ]:
library(car)
library(TSA)
library(tseries)
library(lmtest)
library(fBasics)
library(rgl)
library(forecast)
library(openxlsx)
library(timsac)
library(strucchange) 
library(sandwich) 
library(nlme)
library(foreign)
library(gplots)
library(plm)

##Preparación de los datos

###Importamos nuestros datos

In [ ]:
path = "https://raw.githubusercontent.com/mogollonalex/data/main/datosec.txt"
datos <-read.table(path,  header=TRUE, sep=";", na.strings="NA", dec=".", strip.white=TRUE)
datos

### Extracción de nuestros datos para transformadorlos en series de tiempo

In [ ]:
year <-datos$year
year
PIBpc <-datos$PIBpc
PIBpc
GCF <-datos$GCF
GCF
GCFG <- datos$GCFG
GCFG
FBK <- datos$FBK
FBK

In [ ]:
year=ts(year, start=1990, frequency=1)
year
PIBpc=ts(PIBpc, start=1990, frequency=1)
PIBpc
GCF=ts(GCF, start=1990, frequency=1)
GCF
GCFG=ts(GCFG, start=1990, frequency=1)
GCFG
FBK=ts(FBK, start=1990, frequency=1)
FBK

##Regresión lineal multiple

In [ ]:
###Modelo de regresión múltiple
mod1=lm(PIBpc~GCF+GCFG+FBK, x=T)
summary(mod1)

##Pruebas estadísticas

###Normalidad de residuos

In [ ]:
#Normalidad
res=residuals(mod1)
shapiro.test(res)
AIC(mod1)
BIC(mod1)


###Matríz de correlación

In [ ]:
####Matriz de correlaciones simples
m=data.frame(PIBpc,GCF,GCFG,FBK)
m

In [ ]:
#Correlacion
cor(m)

###Multicolinealidad

In [ ]:
#Multicolinealidad
vif(mod1)

###Heterocedásticidad

In [ ]:
#Pruebas para detectar heterocedásticidad 
# Prueba de Breush-Pagan-Godfrey (BPG)
bptest(mod1)

###Autocorrelación

In [ ]:
#Pruebas para detectar autocorrelacionados. 
# Test de Breush Godfrey
bgtest(mod1, order = 2, order.by = NULL, type = c("Chisq"),
data = datos)

###Error de especificación

In [ ]:
# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)

##Corrección de problemas Econometrícos

###Corregir autocorrelación de residuos
a) modelo en diferencia, b) agregar variable de tendencia

a) Variables en diferencia

In [ ]:
################
# Para corregir autocorreción 
# a) modelo en diferencia, b) agregar variable de tendencia

# a) Variables en diferencia

dPIBpc = diff(PIBpc)
dPIBpc

dGCF = diff(GCF)
dGCF

dGCFG = diff(GCFG)
dGCFG

dFBK = diff(FBK)
dFBK

###Modelo de regresión múltiple
mod1=lm(dPIBpc~dGCF+dGCFG+dFBK, x=T)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

#Pruebas para detectar autocorrelacionados. 
# Test de Breush Godfrey
bgtest(mod1, order = 2, order.by = NULL, type = c("Chisq"),
data = datos)

# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)



b) agregar variable de tendencia

In [ ]:
# b) Agregar variable de tendencia

# modelo

###Modelo de regresión múltiple
mod1=lm(PIBpc~GCF+GCFG+FBK+year, x=T)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

#Pruebas para detectar autocorrelacionados. 
# Test de Breush Godfrey
bgtest(mod1, order = 2, order.by = NULL, type = c("Chisq"),
data = datos)

# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)



###Corregir Heterocedasticidad de residuos

a) MCG

In [ ]:
################
# Para corregir heterocedasticidad
# a) MCG, b) Aplicar transformación logarítmica

# a) MCG


###Modelo de regresión múltiple


mod1= gls(PIBpc~GCF+GCFG+FBK)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)



b) Aplicar transformación logarítmica

In [ ]:
# b) Aplicar transformación logarítmica

#Variables en logaritmo

lPIBpc = log(PIBpc)
lPIBpc

lGCF = log(GCF)
lGCF

lGCFG = log(GCFG)
lGCFG

lFBK = log(FBK)
lFBK

###Modelo de regresión múltiple
mod1=lm(lPIBpc~lGCF+lGCFG+lFBK, x=T)
summary(mod1)

#Normalidad
res=residuals(mod1)
shapiro.test(res)

#Multicolinealidad
vif(mod1)

#Pruebas para detectar heterocedásticidad 
# Prueba de Breush-Pagan-Godfrey (BPG)
bptest(mod1)

# Error de especificación
##Test de Ramsey-Reset
resettest(mod1)

In [ ]:
print("Hola mundo!")

In [ ]:
sprintf("la suma de a + b es %s,y la de a-b es %s",a+b, 